# Learning to Drive in Adverse Weather Conditions

## Pre-Notes

- Only got access to lab computers on 04/11/21 (Week 8)
- Had to update version of Firefox as github would not work correctly
- No sudo access so could only install things that were packaged or had their own installer

## Week 1

### Setup Carla & Anaconda

- Carla 0.9.12 was installed on the machines by IT in /opt little setup required just figuring out configs and some crashing errors
- Anaconda was installed from a package, Quite easy and quick 
- Issues installing Pytorch into env

### Install Carla Gym

- Installing Carla Gym not too hard, API not very well documented so had dig through code files
- Some Issues as pygame only works with python <= 3.7, overcame this with anaconda env (there is a 3.8 version but it requires a sudo install)
- Had to update some code do to older version of Carla being used (See Adapt Carla Env)

### Additional Libraries

- interp e2e driving installed fine but crashed after a number of runs due to low memory (Had to remove Logging as it was for TF 1.0)
- d4rl would not install due to mujuco dependency (May be free now)
- didn't try d3rlpy ye

### Adapt Carla Env

- Fix not deleting sensors correctly bug
- Lidar Location errorhad to be fixed
- Change Reward function the env return

### Alter Pytorch Tutorial

- Reached out to author of Latent Space driving paper and was recommended to follow [this](https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html) tutorial
- Began tyring to get an understanding of the code and what it was doing
- Changed the input from being gotten from getscreen() to using the returned image data from gym-env

## Week 2

### First Experiment - Minimize Distance to Waypoint

- Recommed to start with an easier setup using minimum distance to waypoint
- Used Town04 as it has more wide roads and less severe turns to start, 
- Changed carla env to output currentl vehicle states: (position relative to new waypoint, acceleration, angular velocity, previous steering angle)
- Used Reward from Latent Space Paper
- Implemented a single layer dense network
- Implemented some logging of the data (Loss, avg_reward (removed) and lenght of episode) and view with tensorboard
- Error: Was using absolute x,y position but changed to distance and angle from current way point instead (This changed input from 8,1 vec to 9,1 vec
- Ran 100 and 500 episodes in the above model but results were poor and seemed to have little improvement
- Changed archtechture to have 3 dense layers (9,15)(15,9)(9,3) this showed an immediate large improvment in perfromance
- Agent was able to drive on straights and less severe turns quite well but has issues making sharp turns as it tries to premeptively turn to the waypoint.

### Week 3
### First Exp ctd, Carla env and 2nd Exp - Minimize Distance to Waypoint with CL

- Started Route Implementation
- Fixed Speed issue 
- Catestrophic Forgetting happening after target net update
- Ran for 1000 steps and had forgetting at step 256
- Running changing target update to 20 and running for 100 did not improve results (Model 6)
- Running changin target update to 50 and running for 100, as of step 400 best performing model so far
    - avg loss is about 0.03
- Start Fix Routes (See if route generation is deterministic and only need to specify starting poin ???)
    - Picked 20 routes with different levels of diffcult (5 Easy, 2 Medium Easy, 6 Medium, 3 Medium Hard and 4 Hard)
    - Easy -> Straight
    - Medium Easy -> Straight into slight turn
    - Medium -> Medium Turn 
    - Medium Hard -> Straight into Hard turn
    - Hard -> Hard Turn
- Running first test for 500 episodes using set routes, keep eps lenght = 250, test adam opt with lr = 0.00025 (previous = 0.01), target update at 50 (Could not get to lower loss as much higher level of loss). Model forgot how to drive straight, Could only turn left or right.

### Week 4

- Set up CL learning so would change level or routes every 100 steps
- Checkpointing best model from run
- Forked gym-carla properly to share and pushed alterations
- Testing with CL with larger replay memory of 15000, training slows down a lot.
    - Upping batch size to conteract good training sparisty
    - Loss fn doesn't seem to work correctly any more wiht sparser replay memory
- Did out plan for semester to

### Week 5 (06/01/22)
- Added bigger network, didn't seem to have massive impact, might be problem with eps not working and no exploartion happening for harder ones
- Fixed testing implementation, no step function
- Tried loading previous model and training only on harder tracks, will need to try again for longer time

### Week 6 (17/01/22)

- Trained larger model on just medium and hard tracks but was having a lot of issues getting it to make most of the turns.
- Changed hard routes to inculde 2 90 turns that are part of the road (not an intersetion) and model 11 can make the turn with relative ease. So it works it the situation as described in the latent space paper but is unable to make turns at intersections ? Possibly due to positioning of waypoints and slightly tighter turning radius. Will bring up at next meeting.
- Note: way point may be in the middle of intersection pulling the car out too far and 0.3 turn at each point may be to small to complete turn effectivly.
- Onto visual next !

### Week 7 (24/01/22)
- Wrote Data Collector Class, Can't display semantic and collect tag data at the same time, Will probably need to add another camera.
- Let data collection happen with random agent over 20 random routes, felt this would give the best selection of pictures.
- Started Building autoencoder and need to finish the data loader for the images
- Image set from 18:08 onwards is from model 10 rather than random action
- Saved image matrixs were incorrect so I have downsampled and resaved the first 2500 as 128x128 rather than 256x256

### Week 8 
 - trained autoencoder
 - Built pytorch data seta and data loader
 - Started implementing Full CNN and control model

### Week 9 & 10
- Coivd - started report
- tried training cyclegan on google colab but didnt work

###  Week 11
- Trying to train Full model not working
- Tried fixing to single route as in papers
- No improvement in any results 
- Issue may be the additional state vector throwing off the model or model needds to be larger to cope

### Week 12

- Issue is with AE, doesn't capture enough fine detail about image
- Busy week with tests, interviews and assignments
- Started collecting data from different maps to help AE generalize
- AE appears to start overfitting from epoch two based on test data
- Maybe image set is too large
- Start rewrite of Report

### Week 13
- Trying to fix autoencoder
- Tried using different combos of factors:
    - Add generate pixel type frequencies and use as weigths to cross enotrophy
    - change logmax to sigmoid
    - Grouped tags from 22 to 13 as in orignal paper to reduce burden
- Weights seem to have an impact but slow down training
- Sigmoid seem to perform similar but have seen some suggestions that it may perform better when multiple tags are present
- Rereading paper and checking references seems to suggest that it shoul be a VAE instead of an AE but does not match architecture.
- Inputing into categorical cross entrophy funciton wrong, This may be cause but will need more time to see (Changed back to normal model)

# TODOs

### Long Term Goals
- Rebuild Carla Gym
- Implement Perceptron & Control Modules
- Train on altering weather conditions
- Train against GAN
- Meta-Learning ?

### Medium Term Goals
- Start Autoencoder Training
- Plan for Semester 2
- Standardize testing
- Train Perceptron in Standard conditions

### Short Term

- Look up best optimser and lr and loss function
- Video of best results
- Cumlative Reward
- Sharp Turns
- Start CNN training
- Write up plan

### Done
- ~~Stabilize Training~~
- ~~Try Longer run~~
- ~~Add more dense layers~~ 
- ~~Fix speed~~ 
- ~~Change Avg reward to length of episode~~
- ~~Fixed Routes~~
- ~~Setup Checkpoints~~
- ~~Setup forked gym~~